## Writing code to scrape web url (well its a specific code for mdpi) and download pdf related to topic we search

In [1]:
# To install the required packages, run the following command:
!pip install requests beautifulsoup4 selenium webdriver-manager

Defaulting to user installation because normal site-packages is not writeable


In [2]:
# check if we have the selenium package :)
import selenium
print(selenium.__version__)

4.25.0


In [3]:
# initializing libraries
import os
import requests
from bs4 import BeautifulSoup
import re
from urllib.parse import urljoin
import time

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

In [4]:

## function for creating a directory when its not there
def create_folder(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)

## function to download the pdf file with the filename as provided in the request url
def download_pdf(pdf_url, folder):
    response = requests.get(pdf_url)
    if response.status_code == 200:
        # filename = os.path.join(folder, pdf_url.split('/')[-1])
        filename = os.path.join(folder, pdf_url.split('/')[-1].split('?')[0])
        print('filename', filename)
        with open(filename, 'wb') as f:
            f.write(response.content)
        print(f"Downloaded: {filename}")
    else:
        print(f"Failed to download: {pdf_url}")

## function to get the urls of rthe pdf from the page, 
## use beautifulsoup4 to find our tag of interest.
def get_pdfs_from_page(url, folder, driver):
    print('get_pdfs_from_page')
    driver.get(url)
    WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.TAG_NAME, "body")))
    time.sleep(5)
    
    # Scroll to the bottom of the page until no more new content is loaded
    last_height = driver.execute_script("return document.body.scrollHeight")
    print("last_height",last_height)
    last_height = 8093 # manually setting to sort of hack the code to work!
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(3)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    #print('in soup', driver.page_source)
    for a_tag in soup.find_all('a', href=True):
        if '(PDF)' in a_tag.text:
            print('dwn tag ', a_tag)
            pdf_url = urljoin(url, a_tag['href'])
            download_pdf(pdf_url, folder)
        # else: 
            # print('tag ', a_tag)


In [5]:
## function to browse the provided url using selenium, the get the request body text,
## use beautifulsoup4 to find our tag of interest.
def perform_scrapping():
    query = 'Biobased+Polymers'
    base_url = f"https://www.mdpi.com/books/search?q={query}&sort=relevancy&page="

    folder = "document-corpus"
    create_folder(folder)

    # Set up Selenium WebDriver
    chrome_options = Options()
    #chrome_options.add_argument("--headless")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    # driver = webdriver.Chrome(service=ChromeDriverManager().install(), options=chrome_options)
    # driver = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_options)
    # driver = webdriver.Chrome(executable_path=ChromeDriverManager().install(), options=chrome_options)
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=chrome_options)
    page_number = 1
    while True:
        page_url = f"{base_url}{page_number}"
        driver.get(page_url)
        try:
            WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.TAG_NAME, "body")))
            time.sleep(10)

        
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            next_button = soup.find('button', {'data-testid': 'next-button'})
            is_next_disabled = next_button.has_attr('disabled') if next_button else True


            get_pdfs_from_page(page_url, folder, driver)

            if is_next_disabled:
                break
            page_number += 1
        except Exception as e:
            print(f"Failed to retrieve page: {page_url}, Error: {e}")
            break

    driver.quit()


In [6]:
perform_scrapping()

get_pdfs_from_page
last_height 4052
dwn tag  <a class="flex h-full w-full items-center !justify-between rounded bg-inherit" href="https://mdpi-res.com/bookfiles/book/7328/Biobased_Polymers_for_Environmental_Applications.pdf?v=1728522281" target="_blank"> Free Download (PDF) <svg aria-hidden="true" class="iconify iconify--material-symbols" data-testid="open_in_new" height="24" role="img" style="vertical-align: -0.125em;" viewbox="0 0 24 24" width="24" xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink"><path d="M5 21q-.825 0-1.412-.587T3 19V5q0-.825.588-1.412T5 3h7v2H5v14h14v-7h2v7q0 .825-.587 1.413T19 21zm4.7-5.3l-1.4-1.4L17.6 5H14V3h7v7h-2V6.4z" fill="currentColor"></path></svg></a>
filename document-corpus\Biobased_Polymers_for_Environmental_Applications.pdf
Downloaded: document-corpus\Biobased_Polymers_for_Environmental_Applications.pdf
dwn tag  <a class="flex h-full w-full items-center !justify-between rounded bg-inherit" href="https://mdpi-res.com/bookfil